In [1]:
import numpy as np
import pandas as pd
import pickle

# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
import markdown
from markdown.extensions.wikilinks import WikiLinkExtension

import spacy

from pipeline_functions import *

Using TensorFlow backend.
W1210 01:49:52.696559 139934493534016 deprecation_wrapper.py:119] From /home/matteo/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1210 01:49:52.770574 139934493534016 deprecation_wrapper.py:119] From /home/matteo/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1210 01:49:52.779469 139934493534016 deprecation_wrapper.py:119] From /home/matteo/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1210 01:49:53.189839 139934493534016 deprecation_wrapper.py:119] From /home/matteo/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compa

In [3]:
# title 
# file = open("../images/kensho.png", "rb")
# image = file.read()
# header1 = widgets.Image(
#         value=image,
#         format='png',
#         layout=widgets.Layout(width='20%', height='20%', grid_area='header1'),
#         width=30,
#         height=30
# )
header = widgets.HTML(
        value="<b>Wikipedia-fy</b>", # cut-off check later
        layout=widgets.Layout(width='auto', grid_area='header')
)

# defining some widgets
text = widgets.Textarea(
    value='Test Me!',
    placeholder='Type Something',
    description='Text:',
    disabled=False,
    layout=widgets.Layout(width='auto', grid_area='text')
)

button = widgets.Button(description='Process Text',
                       layout=widgets.Layout(width='auto', grid_area='button'),
                       style=widgets.ButtonStyle(button_color='lightblue'))
out = widgets.Output(layout=widgets.Layout(width='auto', grid_area='out'))

def on_button_clicked(_):
    from IPython.display import display, Markdown, clear_output
    from markdown import markdown
    # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        
        # NED
        pred = predict_entity(text.value)
        # preprocess predicted results for display
        pred_text = text_link_preprocess(pred)
        display(Markdown(pred_text))

# linking button and function together using a buhttps://en.wikipedia.org/wiki/Steve_Jobstton's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.GridBox(children=[header, text, button, out],
               layout=widgets.Layout(
               width='100%',
               grid_template_rows='20% 50% 15% 15%',
               grid_template_columns='20% 20% 20% 20% 20%',
               grid_template_areas='''
               ". . header . ."
               "text text text text text"
               ". . button . ."
               "out out out out out"
               '''))

GridBox(children=(HTML(value='<b>Wikipedia-fy</b>', layout=Layout(grid_area='header', width='auto')), Textarea…

In [ ]:
Apple was the brain-child of Steve. It has produced numerous innovations such as Iphone.

In [ ]:
Michael Jordan was a basketball player on the Chicago Bulls.

In [ ]:
Professor Michael Jordan is a statistician. He's at Berkeley.

In [ ]:
good_example1 = "The Sox play in Boston. Their rivals are the Yankees."
good_example2 = "Oxford and Cambridge are two great universities."
good_example3 = "Ford and Toyota are competitors."
#Interestingly enough, if we removed 'former president',Nixon changes
#President Ford alone just gives Ford Motor Company
good_example4 = "President Ford controversially granted a presidential pardon to former president Nixon."
good_example5 = "Professor Michael Jordan is a statistician. He's at Berkeley."
good_example6 = "Michael Jordan was a basketball player on the Chicago Bulls."
# Michael Jordan is one of the best basketball players.
test_text = 'Apple was the brain-child of Steve. It has produced numerous innovations such as  Iphone.'
# Michael Jordan was a basketball player on the Chicago Bulls.

***

In [ ]:
# # defining some widgets
# text = widgets.Text(
#        value='My Text',
#        description='Document')
# # menu = widgets.Dropdown(
# #        options=['red', 'blue', 'green'],
# #        value='red',
# #        description='Color:')
# box = widgets.HBox([text])


# button = widgets.Button(description='Process Text')
# out = widgets.Output()

# def on_button_clicked(_):
#     from IPython.display import display, Markdown, clear_output
#     from markdown import markdown
#     # "linking function with output"
#     with out:
#         # what happens when we press the button
#         clear_output()
        
#         # NED
#         pred = predict_entity(text.value)
        
#         if not pred:
#             # preprocess predicted results for display
#             pred_text = text_link_preprocess(pred)
        
#             display(Markdown(pred_text))
#         else:
#             display(text)
# # linking button and function together using a buhttps://en.wikipedia.org/wiki/Steve_Jobstton's method
# button.on_click(on_button_clicked)
# # displaying button and its output together
# widgets.VBox([box, button, out])

Get dictionary matching wikidata IDs with corresponding page names. Temporary solution.

In [ ]:
# wikidata 
wikidata = pd.read_csv('../data/wikipages_cleaned.csv')
wikidata.dropna(inplace=True)
wikidata.head()

# get corresponding page title for each wikidata id
wikidata_page_title = (wikidata[['wikidata_numeric_id', 'target_page_title']]
                       .groupby('wikidata_numeric_id', as_index=False)
                       .first())

# remove underscores for page titles
# wikidata_page_title['target_page_title'] = wikidata_page_title['target_page_title'].str.replace('_', ' ')
# convert to dictionary
wikidata_page_dict = {}
for row in range(wikidata_page_title.shape[0]):
    wikidata_page_dict[wikidata_page_title.loc[row, 'wikidata_numeric_id']] = wikidata_page_title.loc[row, 'target_page_title']

with open('../data/wiki_id_to_page.pkl', 'wb') as f:
    pickle.dump(wikidata_page_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
doc = nlp('President Ford granted a pardon to former President Nixon')
displacy.render(doc, style='ent', jupyter=True)